In [1]:
#import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sys
from sklearn.model_selection import train_test_split


## Imports

In [83]:
### Pickled data
import pickle
cleaned_data = pd.read_pickle(r"C:\Users\maorb\Classes\Classical_ML\YDATA-kaggle-assignment\data\processed\cleaned_data_Maor.pkl")
X_train = pd.read_pickle(r"C:\Users\maorb\Classes\Classical_ML\YDATA-kaggle-assignment\data\processed\X_train.pkl")
X_test = pd.read_pickle(r"C:\Users\maorb\Classes\Classical_ML\YDATA-kaggle-assignment\data\processed\X_test.pkl")
y_train = pd.read_pickle(r"C:\Users\maorb\Classes\Classical_ML\YDATA-kaggle-assignment\data\processed\y_train.pkl")
y_test = pd.read_pickle(r"C:\Users\maorb\Classes\Classical_ML\YDATA-kaggle-assignment\data\processed\y_test.pkl")
predictions = pd.read_csv(r'C:\Users\maorb\Classes\Classical_ML\YDATA-kaggle-assignment\predictions\predictions.csv')
X_train_fold_0 = pd.read_pickle(r"C:\Users\maorb\Classes\Classical_ML\YDATA-kaggle-assignment\data\processed\X_train_fold_0.pkl")
y_val_fold_3 = pd.read_pickle(r"C:\Users\maorb\Classes\Classical_ML\YDATA-kaggle-assignment\data\processed\y_val_fold_3.pkl")
X_test_1st = pd.read_pickle(r"C:\Users\maorb\Classes\Classical_ML\YDATA-kaggle-assignment\data\processed\X_test_DoNotTouch.pkl")
predictions_1st = pd.read_csv(r'C:\Users\maorb\Classes\Classical_ML\YDATA-kaggle-assignment\data\Predictions\predictionscatboost.csv',
                                header=None)


In [85]:
X_test

,product,campaign_id,webpage_id,user_group_id,gender,age_level,user_depth,city_development_index,var_1,product_category,user_id_ctr,product_ctr,campaign_id_ctr,Day,Hour,Minute,weekday,campaign_duration_hours
292288,C,360936,13787,2,Male,2.0,3.0,2.0,1.0,3,0.037261,0.069039,0.045040,5.0,11.0,56.0,2.0,19.166667
18031,H,359520,13787,3,Male,3.0,3.0,3.0,0.0,4,0.104797,0.069920,0.059076,5.0,14.0,53.0,2.0,91.533333
202102,A,105960,11085,10,Female,4.0,3.0,1.0,1.0,2,0.061523,0.063782,0.068523,6.0,13.0,38.0,3.0,109.633333
227832,C,359520,13787,3,Male,3.0,3.0,3.0,1.0,4,0.024170,0.069039,0.059076,4.0,16.0,40.0,1.0,62.900000
10866,I,118601,28529,2,Male,2.0,3.0,2.0,0.0,4,0.052058,0.063976,0.053211,5.0,6.0,23.0,2.0,57.300000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
325041,H,82320,1734,5,Male,5.0,3.0,2.0,0.0,1,0.028198,0.069920,0.062465,2.0,10.0,11.0,1.0,9.733333
363708,I,118601,28529,2,Male,2.0,3.0,2.0,0.0,4,0.030761,0.063976,0.053211,2.0,15.0,26.0,3.0,53.083333
5006,C,360936,13787,3,Male,3.0,3.0,2.0,1.0,3,0.061523,0.069039,0.045040,2.0,18.0,24.0,6.0,18.383333
255918,H,105960,11085,4,Male,4.0,3.0,2.0,1.0,5,0.088250,0.069920,0.068523,4.0,6.0,48.0,1.0,61.383333


In [50]:
raw_data = pd.read_csv(r"C:\Users\maorb\Classes\Classical_ML\YDATA-kaggle-assignment\data\raw\train_dataset_full.csv")
raw_test = pd.read_csv(r"C:\Users\maorb\Classes\Classical_ML\YDATA-kaggle-assignment\data\raw\X_test_1st.csv")

In [62]:
users_train = set(raw_data['user_id'].unique())
users_test = set(raw_test['user_id'].unique())
campaigns_train = set(raw_data['campaign_id'].unique())
campaigns_test = set(raw_test['campaign_id'].unique())
products_train = set(raw_data['product'].unique())
products_test = set(raw_test['product'].unique())
webpage_train = set(raw_data['webpage_id'].unique())
webpage_test = set(raw_test['webpage_id'].unique())
age_level_train = set(raw_data['age_level'].unique())
age_level_test = set(raw_test['age_level'].unique())
user_group_id_train = set(raw_data['user_group_id'].unique())
user_group_id_test = set(raw_test['user_group_id'].unique())
product_category_1_train = set(raw_data['product_category_1'].unique())
product_category_1_test = set(raw_test['product_category_1'].unique())
user_depth_train = set(raw_data['user_depth'].unique())
user_depth_test = set(raw_test['user_depth'].unique())



# Count how many test users are also in train
common_users = users_test.intersection(users_train)
common_campaigns = campaigns_test.intersection(campaigns_train)
common_products = products_test.intersection(products_train)
common_webpage = webpage_test.intersection(webpage_train)
common_age_level = age_level_test.intersection(age_level_train)
common_user_group_id = user_group_id_test.intersection(user_group_id_train)
common_product_category_1 = product_category_1_test.intersection(product_category_1_train)
common_user_depth = user_depth_test.intersection(user_depth_train)


# Compute the percentage
percentage_users_in_train = len(common_users) / len(users_test) * 100
percentage_campaigns_in_train = len(common_campaigns) / len(campaigns_test) * 100
percentage_products_in_train = len(common_products) / len(products_test) * 100
percentage_webpage_in_train = len(common_webpage) / len(webpage_test) * 100
percentage_age_level_in_train = len(common_age_level) / len(age_level_test) * 100
percentage_user_group_id_in_train = len(common_user_group_id) / len(user_group_id_test) * 100
percentage_product_category_1_in_train = len(common_product_category_1) / len(product_category_1_test) * 100
percentage_user_depth_in_train = len(common_user_depth) / len(user_depth_test) * 100


print(f"✅ {percentage_users_in_train:.2f}% of the test users are also in train")
print(f"✅ {percentage_campaigns_in_train:.2f}% of the test campaigns are also in train")
print(f"✅ {percentage_products_in_train:.2f}% of the test products are also in train")
print(f"✅ {percentage_webpage_in_train:.2f}% of the test webpage are also in train")
print(f"✅ {percentage_age_level_in_train:.2f}% of the test age_level are also in train")
print(f"✅ {percentage_user_group_id_in_train:.2f}% of the test user_group_id are also in train")
print(f"✅ {percentage_product_category_1_in_train:.2f}% of the test product_category_1 are also in train")
print(f"✅ {percentage_user_depth_in_train:.2f}% of the test user_depth are also in train")



✅ 74.48% of the test users are also in train
✅ 100.00% of the test campaigns are also in train
✅ 100.00% of the test products are also in train
✅ 100.00% of the test webpage are also in train
✅ 87.50% of the test age_level are also in train
✅ 85.71% of the test user_group_id are also in train
✅ 100.00% of the test product_category_1 are also in train
✅ 66.67% of the test user_depth are also in train


In [79]:
selected_users = [7716.0, 1035283.0, 65994.0,75976.0,987498.0]
cleaned_data[cleaned_data['user_id'] == 980231]

,session_id,DateTime,user_id,product,campaign_id,webpage_id,user_group_id,gender,age_level,user_depth,city_development_index,var_1,is_click,product_category
171888,500379.0,2017-07-03 06:52:00,980231.0,H,405490,60305,5,Male,5.0,3.0,NaN,0.0,0.0,3
200018,95832.0,2017-07-07 19:40:00,980231.0,C,405490,60305,5,Male,5.0,3.0,NaN,0.0,0.0,3


In [77]:
raw_test.head(10)

,session_id,DateTime,user_id,product,campaign_id,webpage_id,product_category_1,product_category_2,user_group_id,gender,age_level,user_depth,city_development_index,var_1
0,121580,2017-07-03 10:03,352186,H,82320,1734,1,146115.0,2.0,Male,1.0,3.0,4.0,1
1,95831,2017-07-03 14:21,980231,C,405490,60305,3,NaN,6.0,Male,5.0,3.0,NaN,0
2,421806,2017-07-05 17:47,610332,D,404347,53587,1,NaN,3.0,Male,2.0,3.0,1.0,0
3,585403,2017-07-06 11:01,849506,H,118601,28529,5,82527.0,3.0,Male,2.0,3.0,3.0,0
4,496398,2017-07-02 07:50,499495,B,98970,6970,2,NaN,9.0,Female,2.0,3.0,4.0,1
5,340792,2017-07-06 12:05,1138735,H,359520,13787,4,NaN,3.0,Male,2.0,3.0,4.0,0
6,75278,2017-07-07 09:10,470151,C,405490,60305,3,NaN,4.0,Male,3.0,3.0,2.0,0
7,31980,2017-07-05 16:56,538480,E,82320,1734,1,146115.0,3.0,Male,2.0,3.0,3.0,1
8,54082,2017-07-02 15:06,345136,C,405490,60305,3,NaN,2.0,Male,1.0,3.0,NaN,0
9,236163,2017-07-07 08:28,487171,I,118601,28529,4,82527.0,4.0,Male,3.0,3.0,2.0,0


In [63]:
#mean rows of users in train
mean_rows_per_user_train = raw_data.groupby('user_id').size().mean()
mean_rows_per_user_test = raw_test.groupby('user_id').size().mean()
print(f"Mean rows per user in train: {mean_rows_per_user_train:.2f}")
print(f"Mean rows per user in test: {mean_rows_per_user_test:.2f}")

Mean rows per user in train: 2.88
Mean rows per user in test: 1.45


In [56]:
users_train = raw_data['user_id'].unique()
users_test = raw_test['user_id'].unique()
#Check if the users in the test set are a subset of the users in the train set
print(np.all(np.isin(users_test, users_train)))

False


In [65]:
raw_data.head()

,session_id,DateTime,user_id,product,campaign_id,webpage_id,product_category_1,product_category_2,user_group_id,gender,age_level,user_depth,city_development_index,var_1,is_click
0,98528.0,2017-07-04 16:42,7716.0,C,405490.0,60305.0,3.0,NaN,3.0,Male,3.0,3.0,NaN,1.0,1.0
1,589714.0,2017-07-07 07:40,1035283.0,I,118601.0,28529.0,4.0,82527.0,10.0,Female,4.0,3.0,3.0,1.0,0.0
2,478652.0,2017-07-07 20:42,65994.0,H,359520.0,13787.0,4.0,NaN,4.0,Male,4.0,3.0,2.0,0.0,0.0
3,34536.0,2017-07-05 15:05,75976.0,H,405490.0,60305.0,3.0,NaN,3.0,Male,3.0,3.0,3.0,0.0,0.0
4,71863.0,2017-07-06 20:11,987498.0,C,405490.0,60305.0,3.0,NaN,2.0,Male,2.0,3.0,2.0,0.0,0.0


In [17]:
#First value should be the header. Strip header and make it first values in first row
predictions_1st

,0
0,0.0
1,1.0
2,1.0
3,0.0
4,0.0
...,...
60223,1.0
60224,0.0
60225,0.0
60226,0.0


In [308]:
y_val_fold_3.value_counts()[0]/y_val_fold_3.value_counts().sum()    

0.9319820664496002

In [47]:
raw_data.isna().sum()

session_id                  4166
DateTime                    4109
user_id                     4108
product                     4174
campaign_id                 4188
webpage_id                  4157
product_category_1          4201
product_category_2        308235
user_group_id              19319
gender                     19324
age_level                  19309
user_depth                 19322
city_development_index    108137
var_1                       4161
is_click                    4132
dtype: int64

In [124]:
# Fill missing values in 'campaign_duration_days' within each campaign_id group using the mode
X_train['age_level'] = X_train.groupby('user_id', observed=True)['campaign_duration_hours'].transform(
    lambda x: x.ffill().bfill() if not x.mode().empty else x.fillna(0)
)

X_test['campaign_duration_hours'] = X_test.groupby('webpage_id', observed=True)['campaign_duration_hours'].transform(
    lambda x: x.ffill().bfill() if not x.mode().empty else x.fillna(0)
)


In [43]:
X_train.isna().sum()

product                    0
campaign_id                0
webpage_id                 0
user_group_id              0
gender                     0
age_level                  0
user_depth                 0
city_development_index     0
var_1                      0
product_category           0
Day                        0
Hour                       0
Minute                     0
weekday                    0
campaign_duration_hours    0
dtype: int64

In [29]:
categorical_cols = X_train.select_dtypes(include=['category', 'object']).columns
X_train2 = X_train.copy()

for col in categorical_cols:
    mode_value = X_train2.loc[X_train2[col] != "missing", col].mode()[0]
    mask = X_train2[col] == "missing"
    X_train2.loc[mask, col] = mode_value

    

In [30]:
X_test2 = X_test.copy()

for col in categorical_cols:
    mode_value = X_test2.loc[X_test2[col] != "missing", col].mode()[0]
    mask = X_test2[col] == "missing"
    X_test2.loc[mask, col] = mode_value


In [29]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape, predictions.shape, cleaned_data.shape, raw_data.shape

((293307, 14),
 (73327, 14),
 (293307,),
 (73327,),
 (73160, 1),
 (365798, 14),
 (389163, 15))

In [156]:
X_train.isna().sum()

product                        0
campaign_id                    0
webpage_id                     0
gender                         0
age_level                  11626
user_depth                 11630
city_development_index     79202
var_1                         94
product_category               0
Day                            0
Hour                           0
Minute                         0
weekday                        0
campaign_duration_hours        0
dtype: int64

In [225]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape, predictions.shape, cleaned_data.shape, raw_data.shape

((292638, 13),
 (73160, 13),
 (292638,),
 (73160,),
 (73160, 1),
 (365798, 14),
 (389163, 15))

In [506]:
#find how many duplicated rows we have
duplicates = raw_data[raw_data.duplicated()]
duplicates.shape

(19510, 15)

## Feature Engineering

In [ ]:
import pandas as pd
import numpy as np

def feature_generation(df):
    """Generate date/time features and fill missing values in a faster, 
       more scalable way without repeated group-based ffill/bfill.
    """
    df = df.copy()
    
    # If not already a datetime, convert:
    # df['DateTime'] = pd.to_datetime(df['DateTime'])
    
    # -- 1) Create date/time features --
    df['Month'] = df['DateTime'].dt.month
    df['Day'] = df['DateTime'].dt.day
    df['Hour'] = df['DateTime'].dt.hour
    df['Minute'] = df['DateTime'].dt.minute
    df['weekday'] = df['DateTime'].dt.weekday
    
    # -- 2) Drop unnecessary columns --
    df.drop(columns=['DateTime', 'session_id'], inplace=True, errors='ignore')
    
    # -- 3) Make user_id a consistent type --
    # (Strings are often safer keys for merges.)
    df['user_id'] = df['user_id'].astype(str)
    
    # -- 4) Identify columns to fill by median vs. mode --
    #    (You can tune these lists as needed.)
    columns_to_fill_median = ['Month', 'Day', 'Hour', 'Minute', 'weekday']
    columns_to_fill_mode = [
        'product', 'campaign_id', 'webpage_id', 'gender', 
        'age_level', 'user_depth', 'city_development_index', 
        'var_1', 'product_category'
    ]
    
    # Keep only columns that actually exist in df
    columns_to_fill_median = [c for c in columns_to_fill_median if c in df.columns]
    columns_to_fill_mode = [c for c in columns_to_fill_mode if c in df.columns]
    
    # -- 5) Precompute the user-level medians/modes in one pass each --
    if columns_to_fill_median:
        median_df = (
            df.groupby('user_id')[columns_to_fill_median]
            .median()
            .reset_index()
        )
    
    # Mode can be tricky (pandas mode can return multiple values).
    # We'll define a custom aggregator that picks the first mode if multiple modes exist.
    def agg_mode(s):
        m = s.mode(dropna=True)
        return m.iloc[0] if len(m) > 0 else np.nan
        
    if columns_to_fill_mode:
        mode_df = (
            df.groupby('user_id')[columns_to_fill_mode]
            .agg(agg_mode)
            .reset_index()
        )
        
    # -- 6) Merge those statistics back to df --
    # This is usually much more performant than repeated group transforms:
    if columns_to_fill_median:
        df = df.merge(
            median_df, 
            on='user_id', 
            suffixes=('', '_median')
        )
    if columns_to_fill_mode:
        df = df.merge(
            mode_df, 
            on='user_id', 
            suffixes=('', '_mode')
        )
        
    # -- 7) Fill missing values in df using the merged median/mode --
    if columns_to_fill_median:
        for col in columns_to_fill_median:
            df[col] = df[col].fillna(df[f'{col}_median'])
            df.drop(columns=[f'{col}_median'], inplace=True, errors='ignore')
            
    if columns_to_fill_mode:
        for col in columns_to_fill_mode:
            df[col] = df[col].fillna(df[f'{col}_mode'])
            df.drop(columns=[f'{col}_mode'], inplace=True, errors='ignore')
    
    for col in columns_to_fill_median:
        df[col] = df[col].fillna(df[col].median())

    for col in columns_to_fill_mode:
        df[col] = df[col].fillna(df[col].mode()[0])
    
    df.drop(columns=['user_id','user_group_id'], inplace=True, errors='ignore')
            
    return df

# Example usage:
X_train_u = feature_generation(X_train)
X_test_u  = feature_generation(X_test)


Month
Day
Hour
Minute
weekday
product
campaign_id
webpage_id
gender
age_level
user_depth
city_development_index
var_1
product_category
Month
Day
Hour
Minute
weekday
product
campaign_id
webpage_id
gender
age_level
user_depth
city_development_index
var_1
product_category


In [14]:
def fill_missing_with_mode(df: pd.DataFrame, columns: list):
    df = df.copy()
    for column in columns:
        if column in df.columns:
            mode_value = df[column].mode()[0]  # Calculate the mode
            df[column] = df[column].fillna(mode_value)  # Fill missing values with the mode
    return df

def fill_missing_with_median(df: pd.DataFrame, columns: list):
    df = df.copy()
    for column in columns:
        if column in df.columns:
            median_value = df[column].median()  # Calculate the median
            df[column] = df[column].fillna(median_value)  # Fill missing values with the median
    return df   

In [15]:
X_train.columns

Index(['session_id', 'DateTime', 'user_id', 'product', 'campaign_id',
       'webpage_id', 'user_group_id', 'gender', 'age_level', 'user_depth',
       'city_development_index', 'var_1', 'product_category'],
      dtype='object')

In [25]:
def fill_missing_with_mode(df: pd.DataFrame, columns: list):
    df = df.copy()
    for column in columns:
        if column in df.columns:
            mode_value = df[column].mode()[0]  # Calculate the mode
            df[column] = df[column].fillna(mode_value)  # Fill missing values with the mode
    return df

def fill_missing_with_median(df: pd.DataFrame, columns: list):
    df = df.copy()
    for column in columns:
        if column in df.columns:
            median_value = df[column].median()  # Calculate the median
            df[column] = df[column].fillna(median_value)  # Fill missing values with the median
    return df

def determine_categorical_features(df: pd.DataFrame, cat_features: list = None):

    if cat_features:
        cat_features = [col for col in cat_features if col in df.columns]
    else:
        cat_features = df.select_dtypes(include=['object', 'category']).columns.tolist()

    for col in cat_features:
        if col in df.columns:
            # Ensure column is treated as category
            df[col] = df[col].astype("category")

            # Add "missing" only if it's not already a category
            if "missing" not in df[col].cat.categories:
                df[col] = df[col].cat.add_categories("missing")

            # Fill missing values with "missing"
            df[col] = df[col].fillna("missing")

    return cat_features

def feature_generation2(df, use_missing_with_mode=False, get_dumm=False, catb=True, cat_features=None):

    df = df.copy()

    # Generate time-based features
    df['Month'] = df['DateTime'].dt.month
    df['Day'] = df['DateTime'].dt.day
    df['Hour'] = df['DateTime'].dt.hour
    df['Minute'] = df['DateTime'].dt.minute
    df['weekday'] = df['DateTime'].dt.weekday

    # Handle categorical features if `catb` is True
    if catb:
        cat_features = determine_categorical_features(df, cat_features)

    # Fill missing values
    if use_missing_with_mode:
        print("Filling missing values with mode")
        columns_to_fill_mode = ["product", "campaign_id", "webpage_id", "user_group_id", "gender", "age_level", "user_depth", "city_development_index", "var_1", "product_category",
                                "Month", "Day", "Hour"]
        df = fill_missing_with_mode(df, columns_to_fill_mode)

    columns_to_fill_median = ["Month", "Day", "Hour", "Minute", "weekday", "city_development_index", "age_level", "user_depth"]
    df['campaign_id'] = df['campaign_id'].fillna(df['campaign_id'].mode()[0]) #userid and sessionid have nas. What can we do else?
    df['var_1'] = df['var_1'].fillna(df['var_1'].mode()[0])
    df = fill_missing_with_median(df, columns_to_fill_median)

    # Generate campaign-based features
    df['start_date'] = df.groupby('campaign_id', observed=True)['DateTime'].transform('min')
    df['campaign_duration'] = df['DateTime'] - df['start_date']
    df['campaign_duration_days'] = df['campaign_duration'].dt.total_seconds() / (3600*24)
    df['campaign_duration_days'] = df['campaign_duration_days'].fillna(
        df.groupby('campaign_id', observed=True)['campaign_duration_days'].transform(lambda x: x.mode().iloc[0])).astype(int)

    # Drop unnecessary columns
    df.drop(columns=['DateTime', 'start_date', 'campaign_duration', 'session_id', 'user_id', 'user_group_id'], inplace=True)

    # One-hot encoding if `get_dumm` is True
    if get_dumm:
        columns_to_d = ["product", "campaign_id", "webpage_id", "product_category", "gender"]
        df = pd.get_dummies(df, columns=columns_to_d)

    return df


In [26]:
X_train2 = feature_generation2(X_train)
X_test2 = feature_generation2(X_test)
cat_features = determine_categorical_features(X_train2)

KeyError: 'DateTime'

In [31]:
columns_to_d = ["product", "campaign_id", "webpage_id", "product_category", "gender","user_group_id"]
X_train_d = pd.get_dummies(X_train2, columns=columns_to_d)
X_test_d = pd.get_dummies(X_test2, columns=columns_to_d)

In [212]:
X_train2['gender'].value_counts()

gender
Male       248397
Female      32776
missing     11465
Name: count, dtype: int64

In [23]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

rf = RandomForestClassifier(n_estimators=100, random_state=42,class_weight='balanced')
rf.fit(X_train_d, y_train)
y_pred = rf.predict(X_test_d)

f1_score(y_test, y_pred)

0.040829346092503986

In [18]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.metrics import f1_score

# Compute sample weights
sample_weights = compute_sample_weight(class_weight="balanced", y=y_train)

# Train AdaBoost with sample weights
adamodel = AdaBoostClassifier(n_estimators=100, random_state=42, algorithm='SAMME')
adamodel.fit(X_train_d, y_train, sample_weight=sample_weights)

# Make predictions
y_pred = adamodel.predict(X_test_d)

# Evaluate
f1 = f1_score(y_test, y_pred)
print("F1 Score:", f1)


F1 Score: 0.13929004519898577


In [93]:
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

# Apply SMOTE to balance the training set
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train2, y_train)

# Train Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train_smote, y_train_smote)

# Predict and evaluate
y_pred2 = rf.predict(X_test2)



In [46]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix, classification_report
#adjuct the balance of the classes
model = LogisticRegression(max_iter=1000, class_weight='balanced', random_state=42, solver='liblinear',
                           penalty='l2', C=1)
model.fit(X_train_d, y_train)

y_pred_LR = model.predict(X_test_d)
print(f1_score(y_test, y_pred_LR))

0.14371159806427056


In [49]:
from sklearn.model_selection import cross_val_score
model = LogisticRegression(max_iter=1000, class_weight='balanced', random_state=42, solver='liblinear',
                           penalty='l2', C=1)
#run it on all
scores = cross_val_score(model, X_train_d, y_train, cv=5, scoring='f1')
scores

array([0.14191397, 0.14630052, 0.14374937, 0.1396726 , 0.14359774])

In [50]:
np.mean([0.14191397, 0.14630052, 0.14374937, 0.1396726 , 0.14359774])

0.14304684

In [47]:
#import cross_val_score
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB, ComplementNB


model1 = ComplementNB()
model2 = GaussianNB()
model3 = MultinomialNB()
model4 = BernoulliNB()   
print("Complement Bayes: ", cross_val_score(model1, X_train_d, y_train, cv=5, scoring='f1'))
print("Gaussian Bayes: ", cross_val_score(model2, X_train_d, y_train, cv=5, scoring='f1'))
print("Multinomial Bayes: ", cross_val_score(model3, X_train_d, y_train, cv=5, scoring='f1'))
print("Bernoulli Bayes: ", cross_val_score(model4, X_train_d, y_train, cv=5, scoring='f1'))

Complement Bayes:  [0.13291654 0.13741399 0.13816756 0.13567328 0.13666374]
Gaussian Bayes:  [0.13584671 0.14390066 0.13671663 0.13889985 0.13384344]
Multinomial Bayes:  [0.00100806 0.         0.         0.         0.        ]
Bernoulli Bayes:  [0. 0. 0. 0. 0.]


In [30]:
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB, ComplementNB

# Compute sample weights
sample_weights = compute_sample_weight(class_weight="balanced", y=y_train)

# Define models without random_state
model1 = ComplementNB()
model2 = GaussianNB()
model3 = MultinomialNB()
model4 = BernoulliNB()

# Fit models with sample weights
model1.fit(X_train_d, y_train, sample_weight=sample_weights)
model2.fit(X_train_d, y_train, sample_weight=sample_weights)
model3.fit(X_train_d, y_train, sample_weight=sample_weights)
model4.fit(X_train_d, y_train, sample_weight=sample_weights)

# Cross-validation with sample weights
print("Complement Bayes:", cross_val_score(model1, X_train_d, y_train, cv=5, scoring='f1'))
print("Gaussian Bayes:", cross_val_score(model2, X_train_d, y_train, cv=5, scoring='f1'))
print("Multinomial Bayes:", cross_val_score(model3, X_train_d, y_train, cv=5, scoring='f1'))
print("Bernoulli Bayes:", cross_val_score(model4, X_train_d, y_train, cv=5, scoring='f1'))


Complement Bayes: [0.13285236 0.13694722 0.13831141 0.13642458 0.13636068]
Gaussian Bayes: [0.13413932 0.14055185 0.13428364 0.13605852 0.13060411]
Multinomial Bayes: [0. 0. 0. 0. 0.]
Bernoulli Bayes: [0. 0. 0. 0. 0.]


In [33]:
#import sgdclassifier
from sklearn.linear_model import SGDClassifier
model = SGDClassifier(random_state=42, loss='log_loss', class_weight='balanced')
model.fit(X_train_d, y_train)

# Predict and evaluate
y_pred = model.predict(X_test_d)
f1 = f1_score(y_test, y_pred)
print(f"F1-score: {f1}")

F1-score: 0.14070554937834834


In [44]:
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.kernel_approximation import RBFSampler,Nystroem
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

# Load CTR data (Assume df is the CTR dataset)


# Define the full pipeline
rbf_pipeline = Pipeline([
    ("rbf_feature_map", Nystroem(gamma=1, n_components=300)),  # RBF feature map
    ("sgd", SGDClassifier(random_state=42, loss='log_loss', class_weight='balanced'))  # SGD with log-loss
])

# Split the data


# Train the model
rbf_pipeline.fit(X_train_d, y_train)

# Predict
y_pred_rbf = rbf_pipeline.predict(X_test_d)

# Evaluate
f1_rbf = f1_score(y_test, y_pred_rbf)
print(f"F1-score with RBF SGD: {f1_rbf}")


F1-score with RBF SGD: 0.12733747750277624


**Baseline? :()**

In [163]:
cleaned_data['is_click'].value_counts()[0]/cleaned_data['is_click'].value_counts().sum()

0.9323260378678943

In [6]:
a = 1- cleaned_data['is_click'].value_counts()[0]/cleaned_data['is_click'].value_counts().sum()

In [569]:
X_train1 = X_train.drop(columns=['session_id', 'DateTime', 'user_id'])

In [309]:
import logging
import numpy as np
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold
from catboost import CatBoostClassifier

# Configure logger
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def objective(trial):
    # Define hyperparameters to optimize
    params = {
        "iterations": 1000,
        "depth": trial.suggest_int("depth", 4, 8),
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1, 100),
        "grow_policy": trial.suggest_categorical("grow_policy", ["SymmetricTree", "Depthwise", "Lossguide"]),
        "bootstrap_type": trial.suggest_categorical("bootstrap_type", ["Bayesian", "Bernoulli"]),
        "class_weights": [1, 1 / 0.06767396213210575],  # Fixed class weights
        "eval_metric": "F1",
        "early_stopping_rounds": 100,
        "random_seed": 42,
        "verbose": 0,
    }

    # Add bagging_temperature or subsample based on bootstrap_type
    if params["bootstrap_type"] == "Bayesian":
        params["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0.0, 1.0)
    elif params["bootstrap_type"] == "Bernoulli":
        params["subsample"] = trial.suggest_float("subsample", 0.5, 1.0)

    # Initialize the model
    model = CatBoostClassifier(**params)

    # Cross-validation
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    scores = []

    for fold_index, (train_idx, val_idx) in enumerate(skf.split(X_train, y_train)):
        # Create train-validation splits
        X_train_cv, X_val_cv = X_train.iloc[train_idx], X_train.iloc[val_idx]
        y_train_cv, y_val_cv = y_train.iloc[train_idx], y_train.iloc[val_idx]

        # Check if both classes are present in the training and validation sets
        if len(np.unique(y_train_cv)) < 2 or len(np.unique(y_val_cv)) < 2:
            logger.warning(f"Fold {fold_index}: Skipping due to only one class in y_train_cv or y_val_cv")
            continue

        # Train the model
        model.fit(
            X_train_cv,
            y_train_cv,
            cat_features=cat_features,
            eval_set=(X_val_cv, y_val_cv),
            early_stopping_rounds=50,
            use_best_model=True
        )

        # Predict on the validation set
        y_pred_val = model.predict(X_val_cv)

        # Calculate F1 score
        try:
            score = f1_score(y_val_cv, y_pred_val)
            scores.append(score)
            logger.info(f"Fold {fold_index}: F1 score = {score}")
        except Exception as e:
            logger.error(f"Error calculating F1 score on fold {fold_index}: {e}")
            continue

    # Return the average F1 score across folds
    if scores:
        return np.mean(scores)
    else:
        logger.warning("All folds were skipped. Returning 0.0.")
        return 0.0

In [242]:
def objective(trial):
    params = {
        "iterations": 1000,
        "depth": trial.suggest_int("depth", 3, 5),
        "learning_rate": trial.suggest_float("learning_rate", 0.08, 0.15),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 15, 40),
        "random_strength": trial.suggest_float("random_strength", 0.1, 1.5),
        #"bagging_temperature": trial.suggest_float("bagging_temperature", 0.0, 1.0),
        "grow_policy": trial.suggest_categorical("grow_policy", ["SymmetricTree", "Depthwise"]),
        "bootstrap_type": trial.suggest_categorical("bootstrap_type", ["Bayesian", "Bernoulli", "MVS"]),
        #"class_weights": [1, 1 / trial.suggest_float("class_weight_ratio", 1.0, 10.0)],
        "eval_metric": "F1",
        "class_weights": [1, 1 / 0.06767396213210575],
        "early_stopping_rounds": 100,
        "random_seed": 42,
        "verbose": 0,
    }

    if params["bootstrap_type"] == "Bayesian":
        params["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0.1, 0.8)
    if params['grow_policy'] == 'Depthwise':
        params['min_data_in_leaf'] = trial.suggest_int("min_data_in_leaf", 1, 10)
    elif params["bootstrap_type"] == "Bernoulli":
        params["subsample"] = trial.suggest_float("subsample", 0.6, .9)

    model = CatBoostClassifier(**params)

    X_train_sub, X_val_sub, y_train_sub, y_val_sub = train_test_split(
        X_train,
        y_train,
        test_size=0.2,
        random_state=42,
        stratify=y_train
    )

    model.fit(
        X_train_sub,
        y_train_sub,
        cat_features=cat_features,
        eval_set=(X_val_sub, y_val_sub),
        early_stopping_rounds=50,
        use_best_model=True
    )

    y_pred_val = model.predict(X_val_sub)
    f1 = f1_score(y_val_sub, y_pred_val)
    return f1





In [243]:
# Create study that aims to maximize F1
import optuna
study = optuna.create_study(direction="maximize")

# Optimize over 'objective' for a certain number of trials
study.optimize(objective, n_trials=100, show_progress_bar=True)


[I 2025-01-27 22:58:00,203] A new study created in memory with name: no-name-5758c482-6afb-4363-a73d-0de97e7ba98c


  0%|          | 0/100 [00:00<?, ?it/s]

[I 2025-01-27 22:58:12,512] Trial 0 finished with value: 0.1472231158424239 and parameters: {'depth': 3, 'learning_rate': 0.11179495588783098, 'l2_leaf_reg': 30.860280395620038, 'random_strength': 0.30182868854996703, 'grow_policy': 'SymmetricTree', 'bootstrap_type': 'MVS'}. Best is trial 0 with value: 0.1472231158424239.
[I 2025-01-27 22:58:21,945] Trial 1 finished with value: 0.14491287240022485 and parameters: {'depth': 5, 'learning_rate': 0.13055535934283002, 'l2_leaf_reg': 21.524094448381117, 'random_strength': 1.015927175562968, 'grow_policy': 'Depthwise', 'bootstrap_type': 'Bayesian', 'bagging_temperature': 0.1818611010446386, 'min_data_in_leaf': 7}. Best is trial 0 with value: 0.1472231158424239.
[I 2025-01-27 22:58:31,078] Trial 2 finished with value: 0.14444444444444443 and parameters: {'depth': 4, 'learning_rate': 0.08194672112755871, 'l2_leaf_reg': 16.280719200709708, 'random_strength': 0.6647458591392498, 'grow_policy': 'Depthwise', 'bootstrap_type': 'MVS', 'min_data_in_le

In [245]:
params2 = study.best_params

In [236]:
params2

{'depth': 4,
 'learning_rate': 0.12751986192358583,
 'l2_leaf_reg': 28.56605893525792,
 'random_strength': 1.4329403288787461,
 'grow_policy': 'SymmetricTree',
 'bootstrap_type': 'Bayesian',
 'bagging_temperature': 0.31033906089109137}

In [192]:
params =  study.best_params

In [193]:
params

{'depth': 4,
 'learning_rate': 0.1264579533008154,
 'l2_leaf_reg': 30.03825012048619,
 'grow_policy': 'SymmetricTree',
 'subsample': 0.7558041984536977}

In [238]:
completed_trials = [trial for trial in study.trials if trial.state == optuna.trial.TrialState.COMPLETE]

# Sort the trials by the value (objective) in descending order (maximize performance)
sorted_trials = sorted(completed_trials, key=lambda trial: trial.value, reverse=True)

# Extract the top 5 parameter sets
top_5_params = [trial.params for trial in sorted_trials[:5]]

In [239]:
top_5_params

[{'depth': 4,
  'learning_rate': 0.12751986192358583,
  'l2_leaf_reg': 28.56605893525792,
  'random_strength': 1.4329403288787461,
  'grow_policy': 'SymmetricTree',
  'bootstrap_type': 'Bayesian',
  'bagging_temperature': 0.31033906089109137},
 {'depth': 4,
  'learning_rate': 0.1248507739152174,
  'l2_leaf_reg': 24.826390939702126,
  'random_strength': 1.4094504738081457,
  'grow_policy': 'SymmetricTree',
  'bootstrap_type': 'Bayesian',
  'bagging_temperature': 0.3773129429403069},
 {'depth': 4,
  'learning_rate': 0.10889824051169908,
  'l2_leaf_reg': 24.326419520439615,
  'random_strength': 1.4322494067371123,
  'grow_policy': 'SymmetricTree',
  'bootstrap_type': 'Bayesian',
  'bagging_temperature': 0.481893039827185},
 {'depth': 4,
  'learning_rate': 0.1178823321005181,
  'l2_leaf_reg': 22.111997922334616,
  'random_strength': 1.428924662511862,
  'grow_policy': 'SymmetricTree',
  'bootstrap_type': 'Bayesian',
  'bagging_temperature': 0.4705315566363708},
 {'depth': 4,
  'learning_ra

In [246]:
# Import necessary libraries
import pandas as pd
import logging
from pathlib import Path
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold

# Define the ModelTrainer class
class ModelTrainer:
    def __init__(self, data_dir: False, model_name: str = "catboost", cat_features: list = None):
        #self.data_dir = Path(data_dir)
        self.model_name = model_name
        self.cat_features = cat_features

        logging.basicConfig(level=logging.INFO)
        self.logger = logging.getLogger(__name__)

    def load_data(self):
        self.logger.info(f"Loading preprocessed data from {self.data_dir}...")
        X_train = pd.read_pickle(self.data_dir / "X_train.pkl")
        y_train = pd.read_pickle(self.data_dir / "y_train.pkl").squeeze()
        return X_train, y_train

    def determine_categorical_features(self, X_train: pd.DataFrame):
        if self.cat_features:
            cat_features = [col for col in self.cat_features if col in X_train.columns]
        else:
            cat_features = X_train.select_dtypes(include=['object', 'category']).columns.tolist()

        self.logger.info(f"Categorical features: {cat_features}")
        return cat_features


    def cross_validate_model(self, X_train: pd.DataFrame, y_train: pd.Series, cat_features: list, cv: int = 5):
        if self.model_name == 'catboost':
            model = CatBoostClassifier(
                random_seed=42, verbose=0, eval_metric='F1',
                cat_features=cat_features, class_weights=[1, 10]
            )
        else:
            raise ValueError(f"Unsupported model: {self.model_name}")

        self.logger.info(f"Performing {cv}-fold cross-validation...")
        skf = StratifiedKFold(n_splits=cv, shuffle=True, random_state=42)

        fold_scores = []
        for fold, (train_idx, val_idx) in enumerate(skf.split(X_train, y_train)):
            self.logger.info(f"Processing fold {fold + 1}...")

            X_fold_train, X_fold_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
            y_fold_train, y_fold_val = y_train.iloc[train_idx], y_train.iloc[val_idx]

            self.logger.info(f"Validation set shape for fold {fold + 1}: {X_fold_val.shape}")

            model.fit(X_fold_train, y_fold_train, eval_set=(X_fold_val, y_fold_val), use_best_model=True)

            fold_score = model.best_score_['validation']['F1']
            fold_scores.append(fold_score)

            self.logger.info(f"Fold {fold + 1} F1 score: {fold_score}")

        mean_cv_score = sum(fold_scores) / len(fold_scores)
        self.logger.info(f"Mean cross-validation F1 score: {mean_cv_score}")
        return mean_cv_score

    def train_model(self, X_train: pd.DataFrame, y_train: pd.Series, cat_features: list, val_size: float = 0.2):
        if self.model_name == 'catboost':
            model = CatBoostClassifier(
            random_seed=42,
            verbose=100,
            eval_metric='F1',
            loss_function='Logloss',
            cat_features=cat_features,
           #auto_class_weights='Balanced',
            #bootstrap_type= "Bernoulli",
            #grow_policy= "SymmetricTree",
            class_weights=[1, 1 / a],
            early_stopping_rounds=100,
            **params2
            )

        else:
            raise ValueError(f"Unsupported model: {self.model_name}")

        self.logger.info(f"Training {self.model_name} model...")
        X_train.drop(columns=['session_id', 'DateTime', 'user_id'], inplace=True, errors='ignore')

        X_train_final, X_valid, y_train_final, y_valid = train_test_split(
            X_train, y_train, test_size=val_size, random_state=42)
        

        self.logger.info(f"Training {self.model_name} model with validation set...")
        model.fit(X_train_final, y_train_final, eval_set=(X_valid, y_valid), use_best_model=True)

        return model

# Interactive Workflow for Jupyter Notebook
# Define the data directory
DATA_DIR = "path/to/data"  # Replace with your actual path

# Initialize ModelTrainer
trainer = ModelTrainer(DATA_DIR,model_name="catboost")
cat_features = trainer.determine_categorical_features(X_train)



INFO:__main__:Categorical features: ['product', 'campaign_id', 'webpage_id', 'gender', 'product_category']


In [317]:
X_train.shape

(293307, 15)

In [247]:
model = trainer.train_model(X_train, y_train, cat_features)



INFO:__main__:Training catboost model...
INFO:__main__:Training catboost model with validation set...


0:	learn: 0.5313218	test: 0.5266059	best: 0.5266059 (0)	total: 92.4ms	remaining: 1m 32s
100:	learn: 0.6000101	test: 0.5786546	best: 0.5806488 (93)	total: 9.08s	remaining: 1m 20s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.580648806
bestIteration = 93

Shrink model to first 94 iterations.


In [248]:
from sklearn.metrics import f1_score, classification_report, confusion_matrix
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred, digits=4))

              precision    recall  f1-score   support

         0.0     0.9472    0.5222    0.6732     68404
         1.0     0.0823    0.5954    0.1446      4923

    accuracy                         0.5271     73327
   macro avg     0.5147    0.5588    0.4089     73327
weighted avg     0.8891    0.5271    0.6377     73327



In [249]:
import numpy as np
from sklearn.metrics import f1_score

# After training your CatBoost model:
y_probs = model.predict_proba(X_test)[:, 1]

best_threshold = 0.0
best_f1 = 0.0

# We can search thresholds from 0.0 to 1.0 in small steps
for t in np.linspace(0, 1, 101):
    y_pred = (y_probs >= t).astype(int)
    score = f1_score(y_test, y_pred)
    if score > best_f1:
        best_f1 = score
        best_threshold = t

print(f"Best threshold: {best_threshold}")
print(f"Best F1 score: {best_f1}")

# When predicting on test data, use the best_threshold:
y_probs_test = model.predict_proba(X_test)[:, 1]
y_pred_test = (y_probs_test >= best_threshold).astype(int)


Best threshold: 0.5700000000000001
Best F1 score: 0.15008745080891997


## Naive preprocessing (not my class)

In [19]:
data2= raw_data.drop(columns = ['session_id', 'DateTime', 'user_id','product_category_2'])
data2.dropna(inplace = True) # Just close your eyes and drop the rows with missing values
data2.drop_duplicates(inplace = True) # Drop duplicates

In [23]:
data2.shape

(11879, 11)

In [20]:
X_2 = data2.drop(columns = ['is_click'])
y_2 = data2['is_click']
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X_2, y_2, test_size=0.2, random_state=42)

In [21]:
columns_to_d2 = ["product", "campaign_id", "webpage_id", "user_group_id", "gender", "product_category_1"]

X_train_2_d = pd.get_dummies(X_train_2, columns = columns_to_d2)
X_test_2_d = pd.get_dummies(X_test_2, columns = columns_to_d2)

In [20]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

rf = RandomForestClassifier(n_estimators=100, random_state=42,class_weight='balanced')
rf.fit(X_train_2_d, y_train_2)
y_pred_RF = rf.predict(X_test_2_d)



In [21]:
f1_score(y_test_2, y_pred_RF)

0.14298031865042174

In [22]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix, classification_report

model = LogisticRegression(max_iter=1000, class_weight='balanced')
model.fit(X_train_2_d, y_train_2)

y_pred_LR = model.predict(X_test_2_d)
print(f1_score(y_test_2, y_pred_LR))

0.45353982300884954
